 # Projeto Programação e Algoritmia

## Indice
1.  Introdução
2.  Objetivos
3.  Metodologia
4.  Dados e pre-processamento
5.  Processamento, analise e discussão dos resultados
6. Desenvolvimento de um sistema de recomendação de receitas por meio de ML
7.  Conclusão e notas finais


## Introdução

....

Dieta EAT-Lancet é uma dieta que visa disponibilizar o nivel de nutrição adequado para um ser humano medio.
.....


## Objetivos
O seguinte trabalho visa estudar a composição alimentar de dietas de certos paises, compara-la ao modelo de dieta EAT-Lancet e a composição nutricional de uma lista de receitas do site Food.com, assim como, baseando-se nestes dados, desenvolver um sistema de aprendizagem de maquina capaz de recomendar receitas saudaveis. E para esse fim, sera imprescindivel responder as seguintes questões:


1 - Qual a composição duma dieta me portugal, e em egito, EUA , china e Brasil, e quao diferentes estes sao entre si e entre a EAT-Lancet

2 - Qual a composição de produtos animais nas dietas dos paises citados e a EAT-Lancet?

3 - como se viu a evolução da composição das dietas dos paises citados ao longo dos anos

4 - Existe alguma relação entre a avaliação duma receita e o tamanho das avalições? e o numero de estapas? e o numero de ingridientes? e o tempo de demora?

5 - como se distribuem as receitas filtradas por EAT-Lancet por categoria? Qual categoria possui melhores avaliações?


    perguntas extra ( não tenho certeza se irei incluir ao longo do trabalho):
6 - Quais os produtos mais consumidos em paises e qual a relação com a coltura do pais?
7 - existe alguma diferença notavel de  proporção entre ingredientes de origem animal e os de origem nao animal?
8 - o tempo de preparação da receita influencia de alguma forma o nivel nucticional da receita?
9 - como varia a pegada de carbono das dietas dos diferentes paises ao longo do tempo?

## Metodologia

Tecnologias usadas na elaboração deste trabalho:
    Linguagens de programação: Python;
        Bibliotecas: Seaborn, Matplotlib, numpy, plotly, warning, pandas, sklearn...
        Classes: 
    Softwares: Jupyter (atraves de uma extensão do visual studio code);
    Formatos de ficheiro de dados: csv,parquet(ou feather) e excel (xls);
    Plataforma de hospedagem : github




## Dados e pre-processamento
Como ao longo do trabalho sera necessario comunicar dados entre ficheiros e certas informações ou são irrelevantes ou repetidas, irei tratar de preparar os data frames principais para cada pergunta nesta secssão, assim otimiza-se o processamento em si que vem a seguir e evita-se cria um grande dataframe com dados desnecessarios.

esta secssao tera 3 partes:
1. Preparação do ambiente de trabalho
2. leitura e armazenamento dos dados, otimização e adequação dos tipos de dados  (com uma pre-filtragem)
4. Filtragem



###  Preparação do ambiente de trabalho

In [311]:
import pandas as pd
import chart_studio 
import chart_studio.plotly as py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as ply
import plotly.express  as px
pd.options.plotting.backend = 'plotly'
import warnings
warnings.filterwarnings("ignore")
chart_studio.tools.set_credentials_file(username='Mohaamedl', api_key='MvUpzt7O9SQLxrTEuefC')



# paths dos ficheiros
# filename_diet_animal = '../data/eat-lancet-diet-animal-products.csv' desnecessario visto q esses dados estão ja noutro ficheiro
filename_diet_composition = '../data/eat-lancet-diet-comparison (2).csv'
filename_food_emission = '../data/food-emissions-supply-chain (1).csv'
filename_recipes_raw = 'C:/Users/moham/Documents/data_compressed/RAW_recipes.csv' #! Atenção, deve alterar o caminho para "../data/nome do ficheiro" , usei ficheiros no pc pq nao cabem no github, mas estarei a inclui-los no envio final
filename_reviews_raw= 'C:/Users/moham/Documents/data_compressed/RAW_interactions.csv'

# funções 
def read_diet_and_emission(filename,cols_not_use=[]):
    cols = list(pd.read_csv(filename,nrows=1))
    df = pd.read_csv(filename,usecols=list(col for col in cols if  col not in cols_not_use),dtype={'Entity':'category','Code':'category','Year':'int16'})
    df['Total'] = df.sum(axis=1)
    # reduzir o numero de bits dos float
    datatypes = dict.fromkeys(df.select_dtypes(np.float64).columns, np.float16) 
    df = df.astype(datatypes)
    
    df2 = pd.read_csv(filename) # descomentar para saber quao otimizado ficou
    print(f' De {df2.memory_usage().sum()/1000} Kbytes para {df.memory_usage().sum()/1000} Kbytes, uma redução de {(df2.memory_usage().sum()/1000)/(df.memory_usage().sum()/1000)*100:.2f} % ')
    return df



In [314]:
df_diet = read_diet_and_emission(filename_diet_composition)

df_food_emiss = read_diet_and_emission(filename_food_emission,['Code','Year'])

df_diet.columns = ["País",'Code', "Ano", "cereais", "raízes_tubérculos", "legumes", "frutos", "lacticínios",
            "carne_vermelha", "aves", "ovos", "marisco", "leguminosas", "frutos_de_casca_rija", "óleos", "açúcar",'Total']

df_food_emiss.columns = ['Alimento' , 'utilização_do_solo','exploração_agrícola' ,'alimentação_animal' ,
                'processamento','transporte', 'varejo ','embalagem', 'perdas' ,'Total']
print(df_diet.head(),'\n\n',df_food_emiss.head())

 De 88.324 Kbytes para 30.486 Kbytes, uma redução de 289.72 % 
 De 3.916 Kbytes para 1.293 Kbytes, uma redução de 302.86 % 
          País Code   Ano  cereais  raízes_tubérculos  legumes    frutos  \
0  Afghanistan  AFG  2014   487.50          18.109375  116.875   91.8750   
1  Afghanistan  AFG  2015   478.75          17.453125  115.875  104.0000   
2  Afghanistan  AFG  2016   480.75          17.562500  198.000   84.9375   
3  Afghanistan  AFG  2017   485.75          17.234375  132.500  104.7500   
4      Albania  ALB  2014   413.00         122.750000  699.500  444.2500   

   lacticínios  carne_vermelha       aves       ovos    marisco  leguminosas  \
0      176.375       22.734375   5.425781   4.054688   0.520508     5.781250   
1      157.500       21.265625   5.234375   3.644531   0.575195     6.273438   
2      154.000       19.375000   4.574219   3.644531   0.630371     6.546875   
3      148.500       18.515625   4.191406   4.109375   0.685059     5.781250   
4      930.000     

### leitura e armazenamento dos dados (com uma pre-filtragem)

In [251]:
chart_studio.tools.set_credentials_file(username='Mohaamedl', api_key='MvUpzt7O9SQLxrTEuefC')
cols = list(pd.read_csv(filename_diet_composition,nrows=1))
cols_diet_all =   ["País",'Code', "Ano", "cereais", "raízes_tubérculos", "legumes", "frutos", "lacticínios",
                   "carne_vermelha", "aves", "ovos", "marisco", "leguminosas", "frutos_de_casca_rija", "óleos", "açúcar"]
cols_diet_ani =  ["País", "Ano", "lacticínios",
                   "carne_vermelha", "aves", "ovos", "marisco"]
df_diet = pd.read_csv(filename_diet_composition,usecols=list(col for col in cols if  col not in [])) # por na lista o que não interessar
df_diet
""" 
df_test.columns = cols_diet_all

print(list(col for col in cols if  col not in ('Code')))



def barPlot_diet(df,year,countries=['Portugal']):
    df = df[(df['País'].isin(countries) ) & (df['Ano']==year)] # filtrar os dados por ano e paises
    
    print(df)
    df1 = pd.melt(df, 
        id_vars='País', 
        value_vars=list(df.columns[3:-1]),
        var_name='Tipo', 
        value_name='Consumo(g)'
        ) # converte numa dataframe mais legivel para px, é como um pd.pivot reverso 
    #plot 
    fig = px.bar(df1,x='Consumo(g)',
                y='País',
                color='Tipo',
                text='Tipo',
                barmode='relative',
                title='Composisão de dieta por pais dividida por tipo de alimento'
                )
    

    fig.update_traces(textposition="inside")
    fig.update_traces(visible=True, )
    return fig

countries = ['Portugal','Spain','United States','India','China','Egypt','EAT-Lancet']
countries1 = ['Portugal','EAT-Lancet']
year = 2017
df1= df_test[(df_test['País'].isin(countries))] 
df1['Soma'] = df1.iloc[:,6:11].sum(axis=1)
df_test['Soma'] = df_test.iloc[:,6:11].sum(axis=1)
figs = px.scatter_geo(df_test, locations="Code",
                        color='País',
                        hover_name='País',
                        animation_frame='Ano',
                        size="Soma",
                        projection="natural earth")
fig = barPlot_diet(df_test,year,countries)
fig.show()
figs.show()
df1 """
#py.plot(fig) # para publicar no chart studio



TypeError: read_csv() got an unexpected keyword argument 'cols'